# Import package

In [1]:
from datetime import datetime
from datetime import timedelta
import requests
import numpy as np
import pandas as pd

NCDC CDO web services token: TTyjEBwflgZUPriqiAXQRZrnCgcfPgxD

NWS

In [2]:
start_date = "2023-01-01"
end_date = "2023-04-30"
location_id = "ZIP:10001"  # 紐約市的郵政編碼，這裡以 10001 為例

url = f'https://api.weather.gov/points/38.8894,-77.0352'

response = requests.get(url)

data = response.json()
    
url2 = data['properties']['forecast']
print(url2)

response = requests.get(url2)

if response.status_code == 200:
    data = response.json()
    
    periods = data["properties"]["periods"]
    
    for period in periods:
        name = period["name"]
        startTime = period["startTime"]
        endTime = period["endTime"]
        temperature = period["temperature"]
        temperature_unit = period["temperatureUnit"]
        detailed_forecast = period["detailedForecast"]
        
        # 檢查詳細預報是否包含降雨相關的字詞
        if "rain" in detailed_forecast.lower() or "showers" in detailed_forecast.lower():
            has_rain = True
        else:
            has_rain = False
        
        print(f"Name: {name}")
        print(f"Start Time: {startTime}")
        print(f"End Time: {endTime}")
        print(f"Temperature: {temperature} {temperature_unit}")
        print(f"Detailed Forecast: {detailed_forecast}")
        print(f"Has Rain: {has_rain}")
        print("-------------------")
        
else:
    print("API request failed.")

https://api.weather.gov/gridpoints/LWX/97,71/forecast
Name: Today
Start Time: 2023-07-12T06:00:00-04:00
End Time: 2023-07-12T18:00:00-04:00
Temperature: 94 F
Detailed Forecast: Mostly sunny, with a high near 94. West wind 3 to 7 mph.
Has Rain: False
-------------------
Name: Tonight
Start Time: 2023-07-12T18:00:00-04:00
End Time: 2023-07-13T06:00:00-04:00
Temperature: 74 F
Detailed Forecast: Partly cloudy, with a low around 74. South wind around 7 mph.
Has Rain: False
-------------------
Name: Thursday
Start Time: 2023-07-13T06:00:00-04:00
End Time: 2023-07-13T18:00:00-04:00
Temperature: 96 F
Detailed Forecast: A slight chance of rain showers between 2pm and 5pm, then a slight chance of showers and thunderstorms. Mostly sunny, with a high near 96. South wind 5 to 13 mph, with gusts as high as 21 mph. Chance of precipitation is 20%. New rainfall amounts less than a tenth of an inch possible.
Has Rain: True
-------------------
Name: Thursday Night
Start Time: 2023-07-13T18:00:00-04:00
En

# OpenWeather

https://openweathermap.org/current

key: eefecb7998d70140ab925260f716b4e1

## Geocoding API

https://openweathermap.org/api/geocoding-api

取得紐約經緯度

In [3]:
key = 'eefecb7998d70140ab925260f716b4e1'
city = 'New York'
state = 'NY'
country = 'US'

url = f'http://api.openweathermap.org/geo/1.0/direct?q={city},{state},{country}&appid={key}'

response = requests.get(url)
print(f'Status: {response.status_code}')
data_geo = response.json()

ny_lat = data_geo[0]['lat']
ny_lon = data_geo[0]['lon']
print(f'New York latitude: {ny_lat}')
print(f'New York longitude: {ny_lon}')

Status: 200
New York latitude: 40.7127281
New York longitude: -74.0060152


## Current weather data

In [4]:
key = 'eefecb7998d70140ab925260f716b4e1'
lat = ny_lat
lon = ny_lon
units = 'metric'  # 公制(攝氏)

url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={key}&units={units}'

response = requests.get(url)
print(f'Status: {response.status_code}')
data_current = response.json()
print(data_current)

Status: 200
{'coord': {'lon': -74.006, 'lat': 40.7143}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 23.17, 'feels_like': 23.32, 'temp_min': 19.36, 'temp_max': 25.12, 'pressure': 1013, 'humidity': 68}, 'visibility': 10000, 'wind': {'speed': 3.13, 'deg': 360, 'gust': 4.47}, 'clouds': {'all': 2}, 'dt': 1689159398, 'sys': {'type': 2, 'id': 2008776, 'country': 'US', 'sunrise': 1689154502, 'sunset': 1689208067}, 'timezone': -14400, 'id': 5128581, 'name': 'New York', 'cod': 200}


In [5]:
for i in data_current['main']:
    print(str(i) + ': ', data_current['main'][i])

temp:  23.17
feels_like:  23.32
temp_min:  19.36
temp_max:  25.12
pressure:  1013
humidity:  68


原資料直接取出timestamp會是台灣時間

In [6]:
dt = data_current['dt']
dt = datetime.fromtimestamp(dt)

td = timedelta(hours=12)
dt -= td

print(dt)

2023-07-12 06:56:38


## 5 day weather forecast

https://openweathermap.org/forecast5

未來5天(含今天)的逐3小時預報

In [7]:
key = 'eefecb7998d70140ab925260f716b4e1'
# 經緯度只能抓到小數點後4位
lat = ny_lat
lon = ny_lon
units = 'metric'  # 公制(攝氏)
lang = 'zh_tw'
timezone = 'GMT-4'


url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={key}&units={units}&lang={lang}&timezone=America/New_York'

response = requests.get(url)
print(f'Status: {response.status_code}')
data_5day = response.json()
print(data_5day)

Status: 200
{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1689163200, 'main': {'temp': 23.51, 'feels_like': 23.67, 'temp_min': 23.51, 'temp_max': 25.94, 'pressure': 1013, 'sea_level': 1013, 'grnd_level': 1013, 'humidity': 67, 'temp_kf': -2.43}, 'weather': [{'id': 800, 'main': 'Clear', 'description': '晴', 'icon': '01d'}], 'clouds': {'all': 0}, 'wind': {'speed': 2.1, 'deg': 286, 'gust': 2.41}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2023-07-12 12:00:00'}, {'dt': 1689174000, 'main': {'temp': 25.85, 'feels_like': 26.01, 'temp_min': 25.85, 'temp_max': 30.53, 'pressure': 1013, 'sea_level': 1013, 'grnd_level': 1013, 'humidity': 58, 'temp_kf': -4.68}, 'weather': [{'id': 800, 'main': 'Clear', 'description': '晴', 'icon': '01d'}], 'clouds': {'all': 0}, 'wind': {'speed': 2.5, 'deg': 244, 'gust': 3.38}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2023-07-12 15:00:00'}, {'dt': 1689184800, 'main': {'temp': 29.72, 'feels_like': 29.96, 'temp_min': 29.

In [8]:
# for i in range(40):
#     print(data_5day['list'][i]['dt_txt'])
#     i += 1
print(data_5day['list'][0]['dt_txt'])

2023-07-12 12:00:00


存成DataFrame

In [9]:
dt = {'date_time': [data_5day['list'][i]['dt_txt'] for i in range(40)]}

df_weather_5day = pd.DataFrame(dt)

In [10]:
def add_col_temp(df, feature_name, key):
    '''
    Extracts temperature data from the 'main' section of the JSON file returned by the 5-day weather forecast API,
    and stores it in the specified DataFrame.
    Args:
        df (pandas.DataFrame): The DataFrame where the temperature data will be stored.
        col_name (str): The name of the column in the DataFrame to store the temperature data.
        key (str): The specific temperature item to extract from the 'main' section.
    Note:
        This function does not return a value. The results generated will be directly added to the specified DataFrame.
    '''
    new_col = {feature_name: [data_5day['list'][i]['main'][key] for i in range(40)]}
    df[feature_name] = pd.DataFrame(new_col)


add_col_temp(df_weather_5day, 'temp', 'temp')
add_col_temp(df_weather_5day, 'body_temp', 'feels_like')
add_col_temp(df_weather_5day, 'temp_min', 'temp_min')
add_col_temp(df_weather_5day, 'temp_max', 'temp_max')

df_weather_5day.head()

,date_time,temp,body_temp,temp_min,temp_max
0,2023-07-12 12:00:00,23.51,23.67,23.51,25.94
1,2023-07-12 15:00:00,25.85,26.01,25.85,30.53
2,2023-07-12 18:00:00,29.72,29.96,29.72,32.83
3,2023-07-12 21:00:00,31.67,31.98,31.67,31.67
4,2023-07-13 00:00:00,29.50,30.20,29.50,29.50


In [11]:
# weather: Take main, description

def add_col_weather(df, feature_name, key):
    '''
    Extracts temperature data from the 'main' section of the JSON file returned by the 5-day weather forecast API,
    and stores it in the specified DataFrame.
    Args:
        df (pandas.DataFrame): The DataFrame where the temperature data will be stored.
        col_name (str): The name of the column in the DataFrame to store the temperature data.
        key (str): The specific temperature item to extract from the 'main' section.
    Note:
        This function does not return a value. The results generated will be directly added to the specified DataFrame.
    '''
    new_col = {feature_name: [data_5day['list'][i]['weather'][0][key] for i in range(40)]}
    df[feature_name] = pd.DataFrame(new_col)

add_col_weather(df_weather_5day, 'weather_condition', 'main')
add_col_weather(df_weather_5day, 'weather_description', 'description')

df_weather_5day.head(20)

,date_time,temp,body_temp,temp_min,temp_max,weather_condition,weather_description
0,2023-07-12 12:00:00,23.51,23.67,23.51,25.94,Clear,晴
1,2023-07-12 15:00:00,25.85,26.01,25.85,30.53,Clear,晴
2,2023-07-12 18:00:00,29.72,29.96,29.72,32.83,Clouds,晴，少雲
3,2023-07-12 21:00:00,31.67,31.98,31.67,31.67,Clouds,多雲
4,2023-07-13 00:00:00,29.50,30.20,29.50,29.50,Clouds,多雲
5,2023-07-13 03:00:00,27.80,28.76,27.80,27.80,Clouds,多雲
6,2023-07-13 06:00:00,26.71,27.72,26.71,26.71,Clouds,多雲
7,2023-07-13 09:00:00,25.13,25.32,25.13,25.13,Clouds,多雲
8,2023-07-13 12:00:00,26.06,26.06,26.06,26.06,Clouds,多雲
9,2023-07-13 15:00:00,29.05,29.49,29.05,29.05,Clouds,陰，多雲


原資料直接取出dt_txt會是UTC標準時間

In [12]:
lst = []
td = timedelta(hours=4)
for i in range(40):
    dt_utc = data_5day['list'][i]['dt_txt']
    dt_ny = datetime.strptime(dt_utc, '%Y-%m-%d %H:%M:%S') - td
    lst.append(dt_ny)

dt_dict = {'dt': lst}

df_weather_5day['date_time'] = pd.DataFrame(dt_dict)
df_weather_5day.head(20)


,date_time,temp,body_temp,temp_min,temp_max,weather_condition,weather_description
0,2023-07-12 08:00:00,23.51,23.67,23.51,25.94,Clear,晴
1,2023-07-12 11:00:00,25.85,26.01,25.85,30.53,Clear,晴
2,2023-07-12 14:00:00,29.72,29.96,29.72,32.83,Clouds,晴，少雲
3,2023-07-12 17:00:00,31.67,31.98,31.67,31.67,Clouds,多雲
4,2023-07-12 20:00:00,29.50,30.20,29.50,29.50,Clouds,多雲
5,2023-07-12 23:00:00,27.80,28.76,27.80,27.80,Clouds,多雲
6,2023-07-13 02:00:00,26.71,27.72,26.71,26.71,Clouds,多雲
7,2023-07-13 05:00:00,25.13,25.32,25.13,25.13,Clouds,多雲
8,2023-07-13 08:00:00,26.06,26.06,26.06,26.06,Clouds,多雲
9,2023-07-13 11:00:00,29.05,29.49,29.05,29.05,Clouds,陰，多雲
